In [1]:
import pandas as pd
import numpy as np

import os

#os.listdir(".")
#credit = pd.read_csv("Credit_DataSet.csv")

# TODO: READ THE PROCESSED CSV INSTEAD (FOR EXAMPLE WITH CONVERTED CURRENCY)!

credit = pd.read_excel("./Credit/Credit_DataSet.xlsx", sheetname="DATA", skiprows=0)

# Quickly select numeric columns

to_keep = ['NBR_EMPLOYEES', 'COUNTRY_RISK_GROWTH_RATE',
       'COUNTRY_RISK_DIVIDEND_YIELD', 'COUNTRY_RISK_PAYOUT_RATIO',
       'VOLATILITY_30D', 'VOLATILITY_180D', 'PCT_CHG_1_YEAR', 'PCT_CHG_6_M',
       'DIVIDEND_YIELD', 'MARKETCAP', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES',
       'CURRENT_ASSETS', 'EBIT', 'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH',
       'INTEREST_EXPENSES']

X = credit[ to_keep + ["DEFAULT_PROB"] ].dropna()

Y = X["DEFAULT_PROB"].values
X = X[to_keep].values

In [71]:
# READ PREPROCESSED DATA FROM R FOR NN
X_df = pd.read_csv("some_X.csv", usecols=['NBR_EMPLOYEES', 'EMPL_GROWTH',
       'COUNTRY_RISK_GROWTH_RATE', 'COUNTRY_RISK_DIVIDEND_YIELD',
       'COUNTRY_RISK_PAYOUT_RATIO', 'VOLATILITY_30D', 'VOLATILITY_180D',
       'PCT_CHG_1_YEAR', 'PCT_CHG_6_M', 'DIVIDEND_YIELD', 'MARKETCAP',
       'TOTAL_ASSETS', 'TOTAL_LIABILITIES', 'CURRENT_ASSETS', 'EBIT',
       'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH', 'INTEREST_EXPENSES'])
Y_df = pd.read_csv("some_Y.csv", usecols=[1])

import torch
from torch import nn
from torch.autograd import Variable

X = Variable(torch.FloatTensor(X_df.values)).cuda()
Y = Variable(torch.FloatTensor(Y_df.values)).cuda() # Always stays the same

In [ ]:
def select_most_likely(mlp, loss_fn, x_row, y_row):
    candidates = [] # TODO: For every -1 in row in the specific columns, try 0 and 1
    best_candidate = None
    lowest_loss = 100000000000
    
    for candidate in candidates:
        input = candidate # TODO: Convert to one hot encoding
        # Forward the candidate through the network
        y_pred = mlp(input)
        loss = loss_fn(y_pred, y_row)
        if loss < lowest_loss:
            best_candidate = input
    return input

def get_E_step(mlp, loss_fn, X, Y):
    df_list = []
    for x_row, y_row in zip(X.iterrorws(), Y.iterrows()):
        df_list.append(select_most_likely(mlp, loss_fn, x_row, y_row))

    return pd.DataFrame(df_list)


In [69]:


def get_M_step(mlp, loss_fn, X, Y):
    learning_rate = 1e-4
    for t in range(N_iter):
        y_pred = model(X)
        loss = loss_fn(y_pred, Y)
        if (t % (N_iter / 10) == 0): print(loss)
        model.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.data -= learning_rate * param.grad.data
    return mlp

def EM_algorithm():
    # Make an initial model
    N_hidden = 5
    N_iter = 50000
    model = torch.nn.Sequential(
        torch.nn.Linear(X.shape[1], N_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(N_hidden, Y.shape[1]),
    )

    model = model.cuda()
    loss_fn = torch.nn.SoftMarginLoss(size_average=False)
    
    # Init
    
    for (i in range(100)):
        # Get the expected data
        print("Doing iteration {} E step".format(i))
        X = Variable(torch.FloatTensor(get_E_step(model, loss_fn, X_df, Y_df).values)).cuda()
        # TODO: Check if the predicted data is the same still?
        # Train the model for some iterations
        model = get_M_step(mlp, loss_fn, X, Y)
        print("Doing iteration {} M step".format(i))

        # Get new data
        
    
    

Variable containing:
 2612.3718
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 118.1737
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 40.3395
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 24.1766
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 17.2968
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 13.5008
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 11.0976
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 9.4382
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 8.2215
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 7.2899
[torch.cuda.FloatTensor of size () (GPU 0)]



In [61]:
X

Variable containing:
-2.7050e-01 -3.8609e-02 -4.8866e+00  ...  -2.6039e-01 -2.5953e-02  3.3389e-01
-3.4296e-01 -5.3851e-02  2.9936e-01  ...  -2.7932e-01 -8.0263e-02 -8.4846e-02
-3.2822e-01 -3.6055e-02 -4.8866e+00  ...  -2.9783e-01 -2.1485e-01  6.4142e-02
                ...                   ⋱                   ...                
-3.2185e-01 -2.7771e-02  2.9936e-01  ...  -2.5427e-01  1.1513e-02 -2.2298e-01
-3.1000e-01  5.7829e+01  2.9936e-01  ...  -2.5438e-01 -6.2910e-02 -2.3600e-01
 2.8382e-01 -3.1864e-02  1.2225e+00  ...  -2.6706e-01 -6.4205e-02 -2.0394e-01
[torch.FloatTensor of size (3780,19)]